In [ ]:
%matplotlib inline
from obspy import read, UTCDateTime, Stream
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
station = "GRW0"
channel = "BHZ"


In [ ]:
event_types = ["VTB", "MP", "gugu_long", "gugu_short", "NN", "ND"] 
traces = []
for typ in event_types:
    for file in glob.glob("events/%s/*.mseed" % typ):
     tr = read(file).select(station=station,channel=channel)[0]
     tr.stats.event_type = typ
     traces.append(tr)
st = Stream(traces=traces)

In [ ]:
def plot_st(st):
    st.normalize()
    fig, ax = plt.subplots(1,1, figsize=(12,12))
    colors = sns.color_palette("Set2", len(event_types))
    first = []
    for i,tr in enumerate(st):
        if not tr.stats.event_type in first:
            label= tr.stats.event_type
            first.append(label)
        else:
            label = None
        plt.plot(tr.times(), tr.data + i, c=colors[event_types.index(tr.stats.event_type)], label=label, lw=1)
    plt.margins(0.01)
    plt.legend(loc="upper center", ncol=3)
    plt.xlabel("Times (s)")
    plt.ylabel("Event #")
    plt.title(tr.id)
    plt.show()

In [ ]:
plot_st(st)

In [ ]:
freqlow=1.0
freqhigh=10.0

st2 = st.copy()
# st2.sort()
st2.detrend("demean")
st2.taper(None, max_length=0.5)
st2.filter("bandpass", freqmin=freqlow, freqmax=freqhigh, corners=8, zerophase=True)
plot_st(st2)